In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


In [22]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [23]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [24]:
df = train

In [25]:
df['location'] = df.location.apply(lambda x: '' if x != x else x.lower())
df.loc[df.location == 'new York, ny','location'] = 'new york'
df.loc[df.location == 'nyc','location'] = 'new york'
df.loc[df.location == 'ny','location'] = 'new york'
df.loc[df.location == 'new york city','location'] = 'new york'
df.loc[df.location == 'us','location'] = 'united states'
df.loc[df.location == 'usa','location'] = 'united states'
df.loc[df.location == 'california','location'] = 'united states'
df.loc[df.location == 'southern california','location'] = 'united states'
df.loc[df.location == 'orange county','location'] = 'united states'
df.loc[df.location == 'u.s.a','location'] = 'united states'
df.loc[df.location == 'ca','location'] = 'united states'
df.loc[df.location == 'san francisco bay area','location'] = 'san francisco'
df.loc[df.location == 'uk','location'] = 'united kingdom'

In [26]:
df.loc[df.location == 'ss','location'] = ''
df.loc[df.location == 'everywhere','location'] = ''
df.loc[df.location == 'earth','location'] = ''
df.loc[df.location == '304','location'] = ''
df.loc[df.location == 'world','location'] = ''
df.loc[df.location == 'worldwide','location'] = ''
df.loc[df.location == '??????','location'] = ''
df.loc[df.location == 'pedophile hunting ground','location'] = ''
df.loc[df.location == 'planet earth','location'] = ''
df.loc[df.location == 'global','location'] = ''
df.loc[df.location == 'in the word of god','location'] = ''
df.loc[df.location == '?','location'] = ''
df.loc[df.location == 'mad as hell','location'] = ''
df.loc[df.location == '??','location'] = ''
df.loc[df.location == '?????','location'] = ''
df.loc[df.location == 'happily married with 2 kids','location'] = ''
df.loc[df.location == 'south','location'] = ''
df.loc[df.location == 'world wide','location'] = ''

In [27]:
df = df.replace(np.nan, '', regex=True)
df['keyword'] = df.keyword.str.replace('%20', '_')
df['keyword'] = df.keyword.str.lower()
df['text'] = df['keyword'] + ' ' + df['location'] + ' ' + df['text']

In [28]:
df.sample(10)

,id,keyword,location,text,target
1450,2091,casualty,,casualty @5SOSFamUpdater social casualty,0
1997,2871,damage,http://twitch.tv/jcmonkey,damage http://twitch.tv/jcmonkey @Drothvader @...,1
4836,6883,mass_murder,by a piano probably.,mass_murder by a piano probably. @yelllowheat...,1
878,1272,blood,"ewa beach, hi","blood ewa beach, hi @Anthxvy runs in the blood",0
6229,8892,snowstorm,"dallas, tx","snowstorm dallas, tx 'Snowstorm' 36'x36' oil o...",0
5833,8334,rubble,made here in detroit,rubble made here in detroit #360WiseNews : Ch...,1
5687,8116,rescued,united states,rescued united states A brave little dog gets ...,0
5203,7430,obliterated,canada,obliterated canada @Silverhusky Shtap! Before...,0
6541,9360,survived,,survived Not even the Berlin wall survived th...,0
3546,5069,famine,panamìá,famine panamìá Top 3:\n* Turn on the Darkness...,0


In [29]:
df = df.drop(['id','keyword','location'], axis=1)

In [30]:
def clean_str(string):
    string = re.sub(r'https?\://\S+', '', string)
    string = re.sub(r'http?\://\S+', '', string)
    string = re.sub(r'@\w*\s', '', string)
    string = re.sub(r'#\w*\s', '', string)
    string = re.sub(r'\d', '', string)
    return string

df['text'] = df['text'].apply(lambda x: clean_str(str(x)))

In [31]:
df['text'].dropna(inplace=True)
df['text'] = [entry.lower() for entry in df['text']]
df['text']= [word_tokenize(entry) for entry in df['text']]
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    df.loc[index,'text_final'] = str(Final_words)

In [32]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df['text_final'],df['target'],test_size=0.33)

In [33]:
Tfidf_vect = TfidfVectorizer(max_features=4000)
Tfidf_vect.fit(df['text_final'])
X_train_Tfidf = Tfidf_vect.transform(X_train)
X_test_Tfidf = Tfidf_vect.transform(X_test)

In [34]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train_Tfidf,y_train)

MultinomialNB()

In [35]:
print("Naive Bayes Accuracy Score on train : ",accuracy_score(Naive.predict(X_train_Tfidf), y_train))
print("Naive Bayes Accuracy Score on test : ",accuracy_score(Naive.predict(X_test_Tfidf), y_test))

Naive Bayes Accuracy Score on train :  0.8523529411764705
Naive Bayes Accuracy Score on test :  0.7946677278153601


In [36]:
SVM = svm.SVC(C=1.0, kernel='sigmoid', degree=3, gamma='scale')
SVM.fit(X_train_Tfidf,y_train)

SVC(kernel='sigmoid')

In [37]:
print("SVM Accuracy Score on test : ",accuracy_score(SVM.predict(X_train_Tfidf), y_train))
print("SVM Accuracy Score on test : ",accuracy_score(SVM.predict(X_test_Tfidf), y_test))

SVM Accuracy Score on test :  0.8345098039215686
SVM Accuracy Score on test :  0.7883008356545961


**Submit kaggle**

In [38]:
df = test

In [39]:
df['location'] = df.location.apply(lambda x: '' if x != x else x.lower())
df.loc[df.location == 'new York, ny','location'] = 'new york'
df.loc[df.location == 'nyc','location'] = 'new york'
df.loc[df.location == 'ny','location'] = 'new york'
df.loc[df.location == 'new york city','location'] = 'new york'
df.loc[df.location == 'us','location'] = 'united states'
df.loc[df.location == 'usa','location'] = 'united states'
df.loc[df.location == 'california','location'] = 'united states'
df.loc[df.location == 'southern california','location'] = 'united states'
df.loc[df.location == 'orange county','location'] = 'united states'
df.loc[df.location == 'u.s.a','location'] = 'united states'
df.loc[df.location == 'ca','location'] = 'united states'
df.loc[df.location == 'san francisco bay area','location'] = 'san francisco'
df.loc[df.location == 'uk','location'] = 'united kingdom'

In [40]:
df.loc[df.location == 'ss','location'] = ''
df.loc[df.location == 'everywhere','location'] = ''
df.loc[df.location == 'earth','location'] = ''
df.loc[df.location == '304','location'] = ''
df.loc[df.location == 'world','location'] = ''
df.loc[df.location == 'worldwide','location'] = ''
df.loc[df.location == '??????','location'] = np.nan
df.loc[df.location == 'pedophile hunting ground','location'] = ''
df.loc[df.location == 'planet earth','location'] = ''
df.loc[df.location == 'global','location'] = ''
df.loc[df.location == 'in the word of god','location'] = ''
df.loc[df.location == '?','location'] = ''
df.loc[df.location == 'mad as hell','location'] = ''
df.loc[df.location == '??','location'] = ''
df.loc[df.location == '?????','location'] = ''
df.loc[df.location == 'happily married with 2 kids','location'] = ''
df.loc[df.location == 'south','location'] = ''
df.loc[df.location == 'world wide','location'] = ''

In [41]:
df = df.replace(np.nan, '', regex=True)
df['keyword'] = df.keyword.str.replace('%20', '_')
df['keyword'] = df.keyword.str.lower()
df['text'] = df['keyword'] + ' ' + df['location'] + ' ' + df['text']

In [42]:
df = df.drop(['id','keyword','location'], axis=1)

In [43]:
def clean_str(string):
    string = re.sub(r'https?\://\S+', '', string)
    string = re.sub(r'http?\://\S+', '', string)
    string = re.sub(r'@\w*\s', '', string)
    string = re.sub(r'#\w*\s', '', string)
    string = re.sub(r'\d', '', string)
    return string

df['text'] = df['text'].apply(lambda x: clean_str(str(x)))

In [44]:
df['text'].dropna(inplace=True)
df['text'] = [entry.lower() for entry in df['text']]
df['text']= [word_tokenize(entry) for entry in df['text']]
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(df['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    df.loc[index,'text_final'] = str(Final_words)

In [45]:
X_test_Tfidf = Tfidf_vect.transform(df['text_final'])

In [46]:
predictions_nb = Naive.predict(X_test_Tfidf)

In [47]:
predictions_nb = pd.DataFrame(predictions_nb, columns=['target'])

In [48]:
test['target'] = predictions_nb['target']

In [50]:
results_nb = test[['id', 'target']]

In [51]:
results_nb.to_csv('/kaggle/working/results_nb.csv', index=False)

In [52]:
predictions_svm = SVM.predict(X_test_Tfidf)

In [57]:
predictions_svm = pd.DataFrame(predictions_svm, columns=['target'])

In [58]:
test['target'] = predictions_svm['target']

In [59]:
results_svm = test[['id', 'target']]

In [60]:
results_svm.to_csv('/kaggle/working/results_svm_1.csv', index=False)